<a href="https://colab.research.google.com/github/JonathanMartignon/DeepLearningIntroduction/blob/main/Tarea1/Ejercicio3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Entrena una red completamente conectada para aproximar la compuerta XOR2

Por: 
- Martiñón Luna Jonathan José

Recordamos que la compuerta XOR ($\oplus$) es:

| $x_1$ | $x_2$ | $y$ |
|-------|-------|-----|
| 0     | 0     | 0   |
| 0     | 1     | 1   |
| 1     | 0     | 1   |
| 1     | 1     | 0   |


Y que es posible aproximar este tipo combinando múltiples LTU conectadas en red. Por ejemplo, es posible llevar a cabo la operación XOR con operaciones OR, AND y NAND en la siguiente ecuación:

$$
x_1 \mathbin{\oplus} x_2 = (x_1 \lor x_2) \land \neg(x_1 \land x_2)
$$

Recordando las compuertas OR:

| $x_1$ | $x_2$ | $y$ |
|-------|-------|-----|
| 0     | 0     | 0   |
| 0     | 1     | 1   |
| 1     | 0     | 1   |
| 1     | 1     | 1   |

y and:

| $x_1$ | $x_2$ | $y$ |
|-------|-------|-----|
| 0     | 0     | 0   |
| 0     | 1     | 0   |
| 1     | 0     | 0   |
| 1     | 1     | 1   |

Por lo que $\neg(x_1 \land x_2)$

| $x_1$ | $x_2$ | $y$ |
|-------|-------|-----|
| 0     | 0     | 1   |
| 0     | 1     | 1   |
| 1     | 0     | 1   |
| 1     | 1     | 0   |

Finalmente obteniendo para $x_1 \mathbin{\oplus} x_2 = (x_1 \lor x_2) \land \neg(x_1 \land x_2)$:

| or | neg and | $y$ |
|----|---------|-----|
| 0  |   1     | 0   |
| 1  |   1     | 1   |
| 1  |   1     | 1   |
| 1  |   0     | 0   |



---

# Librerías

---

In [578]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

----

# Definición de la Arquitectura

----

Creo que me confundí un poco, y a lo mejor no resultó en la mejor implementación. Estuve buscando y encontré que algunos usuarios realizaban clases a parte y después las agregaban, por lo que pensé en hacer lo mismo.

Claro está, que probablemente exista un sobreajuste en tanto que los resultados los devuelve directamente en 1 o 0 (No sé si se valía).

In [579]:
#-----------------------------
# Clase para compuerta or
#-----------------------------

class Step(nn.Module):
  def __init__(self):
    # inicilización del objeto padre, obligatorio
    super(Step, self).__init__()

  def forward(self,x):
      self.result = torch.zeros(x.shape[0],1)
      self.m = x.shape[0]

      for i in range(0,self.m):
          self.result[i] = x[i,0] + x[i,1]

      return self.result
#-----------------------------
# Clase para compuerta and
#-----------------------------

class Nand(nn.Module):
  def __init__(self):
    # inicilización del objeto padre, obligatorio
    super(Nand, self).__init__()

  def forward(self,x):
      self.result = torch.zeros(x.shape[0],1)
      self.m = x.shape[0]

      for i in range(0,self.m):
          self.result[i] = x[i,0] * x[i,1]

      return self.result

#-----------------------------
# Clase para la red
#-----------------------------

class FCN(nn.Module):
    
    # inicializador
    def __init__(self):
        
        # inicilización del objeto padre, obligatorio
        super(FCN, self).__init__()
        
        # tamaño de las capas
        self.I = 2 * 4

        #--------------------
        # definición de capas
        #--------------------
        # La primera es una capa de la compuerta Or 
        self.first = nn.Sequential(
            Step(),
            nn.ReLU()
        )
        
        #La segunda será para la compuerta AND

        self.second = Nand()
        self.third = nn.ReLU()
      

        #La última es para la compuerta XOR

        self.fourth = Nand()
      


    # método para inferencia
    def forward(self, x):

      self.x = x
      self.orx = self.first(self.x)
      self.andx =  self.second(self.x)
      self.Nand = torch.logical_not(self.andx,out=torch.empty(self.andx.shape[0], dtype=torch.int16)).reshape(self.andx.shape[0],-1)

      self.xor = self.fourth(torch.cat((self.orx.view([4, 1]),self.Nand.view([4, 1])), dim=1))
        
      return self.xor

In [580]:
model1 = FCN()
print(model1)

FCN(
  (first): Sequential(
    (0): Step()
    (1): ReLU()
  )
  (second): Nand()
  (third): ReLU()
  (fourth): Nand()
)


In [581]:
x = torch.from_numpy(np.array([[0.,0.],
                               [0.,1.],
                               [1.,0.],
                               [1.,1.]]))
y = model1(x)
print('-----------------------------')
print('x_1 \tx_2 \ty')
print('-----------------------------')
for i in range(y.shape[0]):
    print(f'{x[i,0]}\t{x[i,1]}\t{y[i][0]}')

-----------------------------
x_1 	x_2 	y
-----------------------------
0.0	0.0	0.0
0.0	1.0	1.0
1.0	0.0	1.0
1.0	1.0	0.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:78: UserWarning: An output with one or more elements was resized since it had shape [4], which does not match the required output shape [4, 1].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /pytorch/aten/src/ATen/native/Resize.cpp:19.)


---

# Método 2

---

Estuve investigando un poco y, aprovechándome del inciso en el que se podía utilizar cualquier herramienta de pytorch, encontré precisamente algunos que eran para compuertas lógicas.

Claro que viéndome **aún más flojo**, y reitero, no sé si era válido o el objetivo, encontré la función directa (de pytorch)

Aunque no estoy seguro si cuenta, porque creo que no hay capas.

In [582]:
class Metodo2(nn.Module):
    
    # inicializador
    def __init__(self):
        # inicilización del objeto padre, obligatorio
        super(Metodo2, self).__init__()


    # método para inferencia
    def forward(self, x):
      self.x = x
      self.xor = torch.logical_xor(self.x[:,0],self.x[:,1],
                                   out=torch.empty(self.x.shape[0], dtype=torch.int16))
      return self.xor

In [583]:
model2 = Metodo2()
print(model2)

Metodo2()


In [584]:
y2 = model2(x)
print('-----------------------------')
print('x_1 \tx_2 \ty')
print('-----------------------------')
for i in range(y2.shape[0]):
    print(f'{x[i,0]}\t{x[i,1]}\t{y2[i]}')

-----------------------------
x_1 	x_2 	y
-----------------------------
0.0	0.0	0
0.0	1.0	1
1.0	0.0	1
1.0	1.0	0


# Último método (Basado en la libreta 1e_mnist_fcn)

Platicando con algunos compañeros, observo que hacen uso de nn.Linear(), que a mi parecer lo que hace es $\hat{y}=wx+\beta$ pero no entiendo bien de dónde salen los $w$ o los $\beta$

1) Creamos la arquitectura

In [585]:
class Last_FCN(nn.Module):
    
    # inicializador
    def __init__(self):
        
        # inicilización del objeto padre, obligatorio
        super(Last_FCN, self).__init__()
        
        # tamaño de las capas
        self.I = 2

        #--------------------
        # definición de capas
        #--------------------

        # Existen una entrada de 2 dimensiones
        # Y será una salida de 2 dimensiones también
        self.first = nn.Sequential(
            nn.Linear(2,2),
            nn.Sigmoid()
        )
        # Existen otra entrada de 2 dimensiones
        # Y será una salida de 1 dimensión
        self.second = nn.Sequential(
            nn.Linear(2,1),
            nn.Sigmoid()
        )

    # método para inferencia
    def forward(self, x):

      self.x = x
      self.One = self.first(self.x)
      self.Sec = self.second(self.One)
        
      return self.Sec

2) Presentamos el modelo

In [586]:
modelo = Last_FCN()
modelo

Last_FCN(
  (first): Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
    (1): Sigmoid()
  )
  (second): Sequential(
    (0): Linear(in_features=2, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

3) Hacemos inferencia con datos *de juguete*

In [587]:
# Simulando un tensor con dimensiones iguales
# a nuestras entradas 01,00,11,10
xx = torch.zeros((4,2))

# Utilizando la red
y = modelo(xx)

# Verificando la salida
print(f'{x.shape} => {y.shape}')

torch.Size([4, 2]) => torch.Size([4, 1])


4) Procedemos al entrenamiento

In [588]:

#----------------------
# Entrenamiento
#----------------------

# Honestamente, dado las pocas combinaciones que podíamos tener, no creí
# que fuera necesario un conjunto test. A mi parecer habría un sobre ajuste.
# Es por ello que no lo puse

def train(model, data, y_true, learning_rate=1e-3, epocas=20, muestra=10):

    # historiales
    loss_hist = []
    
    # optimizador por gradiente estocástico descendiente
    opt = optim.Adam(model.parameters(), lr=learning_rate)

    MSE = nn.MSELoss()

    # ciclo de entrenamiento por época
    for epoch in range(epocas):

        # computamos logits
        y_lgts = model(data)
        
        # computamos la pérdida

        loss = MSE(y_lgts, y_true)
        
        # vaciamos los gradientes
        opt.zero_grad()
        
        # retropropagamos
        loss.backward()
        
        # actualizamos parámetros
        opt.step()

        loss_hist.append(loss)

        # imprimimos progreso
        if epoch % muestra == 0:
          print(f'E{epoch:02} '
                f'loss=[{loss}]')

    return loss_hist, acc_hist

Probamos con 20 épocas

In [589]:
# instanciamos un modelo
modelo = Last_FCN()

## Nuestras x
data = torch.tensor([[0.,1.],
                     [0.,0.],
                     [1.,1.],
                     [1.,0.]])
y_true = torch.tensor([[0.],[1.],[1.],[0.]])
# entrenamos
loss_hist, acc_hist = train(modelo,data,y_true,learning_rate=0.08)
y = modelo(data)
print('-----------------------------')
print('x_1 \tx_2 \ty')
print('-----------------------------')
for i in range(y.shape[0]):
    print(f'{x[i,0]}\t{x[i,1]}\t{y[i][0]}')

E00 loss=[0.2614307403564453]
E10 loss=[0.25016480684280396]
-----------------------------
x_1 	x_2 	y
-----------------------------
0.0	0.0	0.4826590120792389
0.0	1.0	0.4800490140914917
1.0	0.0	0.49193698167800903
1.0	1.0	0.4893735349178314


Probamos con 100 épocas

In [590]:
# instanciamos un modelo
modelo = Last_FCN()

## Nuestras x
data = torch.tensor([[0.,1.],
                     [0.,0.],
                     [1.,1.],
                     [1.,0.]])
y_true = torch.tensor([[0.],[1.],[1.],[0.]])
# entrenamos
loss_hist, acc_hist = train(modelo,data,y_true,learning_rate=0.08,epocas=100,muestra=10)
y = modelo(data)
print('-----------------------------')
print('x_1 \tx_2 \ty')
print('-----------------------------')
for i in range(y.shape[0]):
    print(f'{x[i,0]}\t{x[i,1]}\t{y[i][0]}')

E00 loss=[0.2771655321121216]
E10 loss=[0.253320574760437]
E20 loss=[0.2497926652431488]
E30 loss=[0.24504898488521576]
E40 loss=[0.2333054393529892]
E50 loss=[0.20893244445323944]
E60 loss=[0.1868313103914261]
E70 loss=[0.17653951048851013]
E80 loss=[0.1725013256072998]
E90 loss=[0.17071278393268585]
-----------------------------
x_1 	x_2 	y
-----------------------------
0.0	0.0	0.6702086925506592
0.0	1.0	0.6609236001968384
1.0	0.0	0.6653938293457031
1.0	1.0	0.05476263165473938


Probamos con 1000 épocas

In [591]:
# instanciamos un modelo
modelo = Last_FCN()

## Nuestras x
data = torch.tensor([[0.,1.],
                     [0.,0.],
                     [1.,1.],
                     [1.,0.]])
y_true = torch.tensor([[0.],[1.],[1.],[0.]])
# entrenamos
loss_hist, acc_hist = train(modelo,data,y_true,epocas=1000,learning_rate=0.08,muestra=100)

y = modelo(data)
print('-----------------------------')
print('x_1 \tx_2 \ty')
print('-----------------------------')
for i in range(y.shape[0]):
    print(f'{x[i,0]}\t{x[i,1]}\t{y[i][0]}')

E00 loss=[0.25084030628204346]
E100 loss=[0.005242226645350456]
E200 loss=[0.0013153718318790197]
E300 loss=[0.0006740197422914207]
E400 loss=[0.0004220898263156414]
E500 loss=[0.00029351815464906394]
E600 loss=[0.00021778541849926114]
E700 loss=[0.00016889387916307896]
E800 loss=[0.00013524522364605218]
E900 loss=[0.00011096531670773402]
-----------------------------
x_1 	x_2 	y
-----------------------------
0.0	0.0	0.009954786859452724
0.0	1.0	0.9916370511054993
1.0	0.0	0.9896552562713623
1.0	1.0	0.009753783233463764


Con 10,000 épocas

In [592]:
# instanciamos un modelo
modelo = Last_FCN()

## Nuestras x
data = torch.tensor([[0.,1.],
                     [0.,0.],
                     [1.,1.],
                     [1.,0.]])
y_true = torch.tensor([[0.],[1.],[1.],[0.]])
# entrenamos
loss_hist, acc_hist = train(modelo,data,y_true,epocas=10000,learning_rate=0.08,muestra=1000)
y = modelo(data)
print('-----------------------------')
print('x_1 \tx_2 \ty')
print('-----------------------------')
for i in range(y.shape[0]):
    print(f'{x[i,0]}\t{x[i,1]}\t{y[i][0]}')

E00 loss=[0.2849968373775482]
E1000 loss=[0.00016201267135329545]
E2000 loss=[4.627384987543337e-05]
E3000 loss=[2.0212692106724717e-05]
E4000 loss=[1.0322917660232633e-05]
E5000 loss=[5.682119535777019e-06]
E6000 loss=[3.25557402902632e-06]
E7000 loss=[1.908321337396046e-06]
E8000 loss=[1.1334534519846784e-06]
E9000 loss=[6.78610263094015e-07]
-----------------------------
x_1 	x_2 	y
-----------------------------
0.0	0.0	0.00058381212875247
0.0	1.0	0.9993900060653687
1.0	0.0	0.9992448091506958
1.0	1.0	0.0005915439687669277


# Análisis de resultados

Todos los modelos se evaluaron en:
 - 20 épocas
 - 100 épocas
 - 1,000 épocas
 - 10,000 épocas

## Utilizando el gradiente descendiente como optimizador y lr = 0.01

- Utilizando 2 Funciones de activación RELU

  Nuestro mejor modelo fue entrenando 1,000 épocas, con una pérdida de 0.126370787. Y valores predichos de:
$$
\hat{y} = [0.49,0.97,0.49,0.04]
$$

- Utilizando Sigmoide en el primer caso y Relu en el segundo

  Nuevamente nuestro mejor valor aparenta ser con 1,000 épocas con una pérdida de 0.2519682049751282 Con 10,000 épocas se pierde. 
$$
\hat{y} = [0.56,0.50,0.49,0.43]
$$

- Utilizando RELU en el primer caso y Sigmoide en el segundo

  Realmente, en este caso no hay alguno que pueda decir que se trate del mejor, pues no se alcanza a distinguir que los valores medios sean mejores a los límites. En este caso, con 1,000 épocas se pierde totalmente y otorga mismo valor a todas: 0.4837266802787781

- Finalmente se usaron 2 Sigmoides
Realmente no existían valores muy diferentes a la combinación anterior. También se probó un Lr = 0.1, pero tampoco lograba sobresalir

## Utilizando el Adam como optimizador (Lr =0.01):
- 2 ReLu:

  En definitiva, esta combinación no es nada óptima. Todo lo envió a 0, exepto al entrenar 10,000 épocas, donde envió a 0.5 todos los valores 

- Sigmoide y Relu:

  Mismo problema, todo lo envía a 0 (en este caso a partir de 100 épocas)

- 2 Sigmoides:

  Casi un aprendizaje perfecto (eso sí, fue hasta las 10,000 épocas). Obteniendo una pérdida de 1.1293
$$
\hat{y}=[0.0023,0.99,0.990.0026]
$$

Al probar con un lr de 0.08, obteníamos resultados bastante cercanos desde las 1,000 épocas, una tasa de pérdida de: 0.000169.

Mientras que con 10,000 la tasa de pérdida lo decrementaba a: 0.0000006786


## Conclusiones
Podemos concluir que el mejor modelo resulta al utilizar 2 sigmoides como funciones de activación y el método de optimización ADAM, con una taza de aprendizaje del 0.08

Nota: En el notebook sólo se mostrarán los resultados con el método anteriormente mencionado
